# **Setup & Imports**

In [ ]:
!nvidia-smi

In [ ]:
!pip install -U "transformers>=4.44"
!pip install -U "accelerate>=0.33"
!pip install -U "datasets>=2.20"
!pip install -U "evaluate>=0.4"
!pip install "huggingface-hub>=0.34.0,<1.0"
!pip install -U scikit-learn
!pip install -U sacrebleu
!pip install -U gdown

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import glob
import os
import torch
from PIL import Image
import itertools
from datasets import Dataset
import evaluate
import os
import shutil
import json

from transformers import (
    DeiTForImageClassification,
    DeiTImageProcessor,
    TrainingArguments,
    Trainer,
    set_seed,
    pipeline,
    EarlyStoppingCallback
)
from torch.nn import CrossEntropyLoss

# **Data Preparation**

In [ ]:
!gdown 1fr-ZuEeJGi5kE1pA3EzlTvHfr-oN5U0P
!unzip Dataset_DFUs.zip
!rm Dataset_DFUs.zip

In [ ]:
count = glob.glob('Dataset/**/*.png', recursive=True)
print("Dataset amount:", len(count))

In [ ]:
img_name = []
img_label = []

for file in glob.glob('Dataset/**/*.png', recursive=True):
    img_name.append(file)
    img_label.append(int(file.split("/")[-2]))

df_master = pd.DataFrame({
    'img_path': img_name,
    'label': img_label
})

print("Total images loaded:")
print(df_master['label'].value_counts())

X = df_master['img_path']
y = df_master['label']

In [ ]:
df_master.head()

# **Configuration & Preprocessing**

In [ ]:
model_checkpoint = 'facebook/deit-base-distilled-patch16-224'
feature_extractor = DeiTImageProcessor.from_pretrained(model_checkpoint)

id2label = { 0: '0', 1: '1' }
label2id = { '0': 0, '1': 1 }

def transform(example_batch):
    inputs = feature_extractor([Image.open(x) for x in example_batch['img_path']], return_tensors='pt')
    inputs['labels'] = example_batch['label']
    return inputs

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

acc_metric  = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric  = evaluate.load("recall")
f1_metric   = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy":  acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall":    rec_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1":        f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
    }


# **Cross-Validation Training**

In [ ]:
class_weights_array = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y),
    y=y
)

class_weights_tensor = torch.tensor(class_weights_array, dtype=torch.float32)

print(f"Using class weights: {class_weights_tensor}")

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            self.class_weights = class_weights.to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        if self.class_weights is not None and labels is not None:
            loss_fct = CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        else:
            loss = outputs.loss if outputs.loss is not None else super().compute_loss(model, inputs, return_outputs)

        return (loss, outputs) if return_outputs else loss

In [ ]:
N_SPLITS = 5
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

all_metrics = []
all_true_labels = np.array([], dtype=int)
all_predictions = np.array([], dtype=int)
fold_logs = []

In [ ]:
print("--- Visualizing 5-Fold Split Distributions ---")

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n--- FOLD {fold + 1}/{N_SPLITS} ---")

    val_df = df_master.iloc[val_idx]
    val_labels = y.iloc[val_idx]

    val_counts = val_labels.value_counts().sort_index()
    print(f"Validation set distribution:\n{val_counts}\n")

    print(f"Displaying 5 sample validation images for Fold {fold + 1}:")

    sample_df = val_df.sample(min(5, len(val_df)))
    image_paths = sample_df['img_path'].tolist()
    image_labels = sample_df['label'].tolist()

    fig, axs = plt.subplots(1, 5, figsize=(15, 3))

    for i, (path, label) in enumerate(zip(image_paths, image_labels)):
        try:
            img = Image.open(path)
            axs[i].imshow(img)
            axs[i].set_title(f"Label: {label}")
            axs[i].axis('off')
        except FileNotFoundError:
            axs[i].set_title("Image Not Found")
            axs[i].axis('off')

    plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class_weights_tensor = class_weights_tensor.to(device)

**Train the model**

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n--- FOLD {fold + 1}/{N_SPLITS} ---")

    set_seed(42 + fold)

    train_df = df_master.iloc[train_idx]
    val_df   = df_master.iloc[val_idx]

    train_dataset = Dataset.from_pandas(train_df).with_transform(transform)
    val_dataset   = Dataset.from_pandas(val_df).with_transform(transform)

    model = DeiTForImageClassification.from_pretrained(
        model_checkpoint,
        num_labels=2,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )

    model.to(device)

    training_args = TrainingArguments(
        output_dir=f"training_deit_fold_{fold+1}",
        per_device_train_batch_size=64,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=15,
        logging_steps=10,
        learning_rate=3e-5,
        weight_decay=0.01,
        warmup_ratio=0.05,
        save_total_limit=2,
        fp16=torch.cuda.is_available(),
        remove_unused_columns=False,
        metric_for_best_model="f1",
        load_best_model_at_end=True,
        use_mps_device= torch.backends.mps.is_available(),
        optim="adamw_torch",
        report_to="none",
    )

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=feature_extractor,
        class_weights=class_weights_tensor,
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    best_dir = trainer.state.best_model_checkpoint or training_args.output_dir

    export_dir = f"export_deit_fold_{fold+1}"

    os.makedirs(export_dir, exist_ok=True)

    shutil.copytree(best_dir, export_dir, dirs_exist_ok=True)

    feature_extractor.save_pretrained(export_dir)

    torch.save({"id2label": id2label, "label2id": label2id}, os.path.join(export_dir, "label_map.pt"))

    print(f"[Fold {fold+1}] Saved best model to: {export_dir}")

    print(f"[Fold {fold+1}] Best metric (F1):", trainer.state.best_metric)
    print(f"[Fold {fold+1}] Best checkpoint:", trainer.state.best_model_checkpoint)
    print(f"[Fold {fold+1}] Epochs actually trained:", trainer.state.epoch)

    fold_logs.append(trainer.state.log_history)

    log_file_path = os.path.join(export_dir, f"fold_{fold+1}_log_history.json")
    with open(log_file_path, 'w') as f:
        json.dump(trainer.state.log_history, f, indent=4)
    print(f"[Fold {fold+1}] Saved log history to: {log_file_path}")

    print(f"--- Evaluating Fold {fold+1} ---")
    metrics = trainer.evaluate()
    print(metrics)

    predictions_output = trainer.predict(val_dataset)
    fold_preds = np.argmax(predictions_output.predictions, axis=1)

    all_predictions = np.concatenate((all_predictions, fold_preds))
    all_true_labels = np.concatenate((all_true_labels, predictions_output.label_ids))
    all_probs_pos   = np.array([], dtype=float)

    logits = predictions_output.predictions
    exp_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    probs = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    probs_pos = probs[:, 1]

    all_probs_pos = np.concatenate((all_probs_pos, probs_pos))

    del model, trainer, predictions_output
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        torch.mps.empty_cache()

print("\n--- Cross-Validation Complete ---")

# **Results & Analysis for Cross Validation**

## **Helper function to plot the graph**

In [ ]:
def curves_from_history(log_history):
    df = pd.DataFrame(log_history)

    if "loss" in df.columns:
        train_curve = (
            df[df["loss"].notna()]
            .groupby("epoch", as_index=False)["loss"]
            .last()
            .rename(columns={"loss": "train_loss"})
        )
    else:
        train_curve = pd.DataFrame(columns=["epoch", "train_loss"])

    if "eval_loss" in df.columns:
        agg_dict = {
            "eval_loss": ("eval_loss", "last")
        }
        if "eval_accuracy" in df.columns:
            agg_dict["eval_accuracy"] = ("eval_accuracy", "last")
        if "eval_f1" in df.columns:
            agg_dict["eval_f1"] = ("eval_f1", "last")

        eval_curve = (
            df[df["eval_loss"].notna()]
            .groupby("epoch", as_index=False)
            .agg(**agg_dict)
        )
    else:
        eval_curve = pd.DataFrame(columns=["epoch", "eval_loss", "eval_accuracy", "eval_f1"])

    curve = pd.merge(train_curve, eval_curve, on="epoch", how="outer").sort_values("epoch")
    return curve

def smooth_xy(x, y, points=200):
    m = ~np.isnan(x) & ~np.isnan(y)
    x, y = np.asarray(x)[m], np.asarray(y)[m]

    order = np.argsort(x)
    x, y = x[order], y[order]

    uniq_x, uniq_idx = np.unique(x, return_index=True)
    uniq_y = y[uniq_idx]

    if len(uniq_x) < 4:
        x_dense = np.linspace(uniq_x.min(), uniq_x.max(), points)
        y_dense = np.interp(x_dense, uniq_x, uniq_y)
        return x_dense, y_dense

    spline = make_interp_spline(uniq_x, uniq_y, k=3)

    x_dense = np.linspace(uniq_x.min(), uniq_x.max(), points)
    y_dense = spline(x_dense)

    return x_dense, y_dense

fold_curves = [curves_from_history(h) for h in fold_logs]

## Plot the graph

In [ ]:
for i, curve in enumerate(fold_curves, 1):
    plt.figure()

    x_acc, y_acc = smooth_xy(curve["epoch"].values, curve["eval_accuracy"].values)
    plt.plot(x_acc, y_acc, label="Eval accuracy", marker='')

    x_f1, y_f1 = smooth_xy(curve["epoch"].values, curve["eval_f1"].values)
    plt.plot(x_f1, y_f1, label="Eval F1", marker='')

    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.title(f"Fold {i} — Accuracy & F1 per epoch")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
for i, curve in enumerate(fold_curves, 1):
    plt.figure()

    x_tr, y_tr = smooth_xy(curve["epoch"].values, curve["train_loss"].values)
    plt.plot(x_tr, y_tr, label="Train loss", marker='')

    x_ev, y_ev = smooth_xy(curve["epoch"].values, curve["eval_loss"].values)
    plt.plot(x_ev, y_ev, label="Eval loss", marker='')

    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title(f"Fold {i} — Loss per epoch")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
all_eval = []
all_train = []
for c in fold_curves:
    all_eval.append(c[["epoch", "eval_loss", "eval_accuracy", "eval_f1"]].dropna())
    all_train.append(c[["epoch", "train_loss"]].dropna())

avg_eval = (pd.concat(all_eval)
            .groupby("epoch", as_index=False)
            .mean(numeric_only=True))
avg_train = (pd.concat(all_train)
             .groupby("epoch", as_index=False)
             .mean(numeric_only=True))

plt.figure()
x_train_loss, y_train_loss = smooth_xy(avg_train["epoch"], avg_train["train_loss"])
plt.plot(x_train_loss, y_train_loss, label="Avg train loss", marker='')
x_eval_loss, y_eval_loss = smooth_xy(avg_eval["epoch"],  avg_eval["eval_loss"])
plt.plot(x_eval_loss, y_eval_loss, label="Avg eval loss", marker='')
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("Average Loss across folds")
plt.legend(); plt.grid(True, alpha=0.3); plt.show()

plt.figure()
x_eval_acc, y_eval_acc = smooth_xy(avg_eval["epoch"], avg_eval["eval_accuracy"])
plt.plot(x_eval_acc, y_eval_acc, label="Avg eval accuracy", marker='', color='orange')

x_eval_f1, y_eval_f1 = smooth_xy(avg_eval["epoch"], avg_eval["eval_f1"])
plt.plot(x_eval_f1, y_eval_f1, label="Avg eval F1", marker='')

plt.xlabel("Epoch"); plt.ylabel("Score")
plt.title("Average Eval Accuracy & F1 Score across folds")
plt.legend(); plt.grid(True, alpha=0.3)
plt.show()

**Confusion Matrix**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(all_true_labels, all_predictions)
labels = ["Class 0", "Class 1"]

plt.figure(figsize=(4.5, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=True,
    xticklabels=labels,
    yticklabels=labels,
    linewidths=0.5,
    linecolor="white"
)
plt.title("Overall Confusion Matrix (all folds)", pad=12)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.tight_layout()
plt.show()


## Report

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import seaborn as sns

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    precision_recall_curve,
    average_precision_score,
    classification_report,
)


all_metrics      = []
all_true_labels  = []
all_predictions  = []
all_probs_pos    = []

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    fold_dir = f"training_deit_fold_{fold}"
    ckpts = [os.path.join(fold_dir, c) for c in os.listdir(fold_dir) if c.startswith("checkpoint")]
    if not ckpts:
        print(f"No checkpoint found for fold {fold}, skipping.")
        continue

    ckpts.sort(key=lambda p: int(p.split("-")[-1]))
    latest_ckpt = ckpts[-1]
    print(f"--- Fold {fold}: loading {latest_ckpt} ---")

    val_df = df_master.iloc[val_idx].reset_index(drop=True)
    val_dataset = Dataset.from_pandas(val_df).with_transform(transform)

    model = DeiTForImageClassification.from_pretrained(latest_ckpt)

    eval_args = TrainingArguments(
        output_dir="tmp_eval",
        per_device_eval_batch_size=16,
        remove_unused_columns=False,
        report_to="none",
    )
    eval_trainer = Trainer(
        model=model,
        args=eval_args,
        data_collator=collate_fn,
    )

    preds_output = eval_trainer.predict(val_dataset)
    logits = preds_output.predictions
    y_true = preds_output.label_ids
    y_pred = np.argmax(logits, axis=1)

    logits_shifted = logits - np.max(logits, axis=1, keepdims=True)
    exp_logits = np.exp(logits_shifted)
    probs = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    probs_pos = probs[:, 1]

    all_true_labels.append(y_true)
    all_predictions.append(y_pred)
    all_probs_pos.append(probs_pos)

    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label=1)
    rec  = recall_score(y_true, y_pred, pos_label=1)
    f1   = f1_score(y_true, y_pred, pos_label=1)

    all_metrics.append({
        "fold": fold,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
    })

all_true_labels = np.concatenate(all_true_labels, axis=0)
all_predictions = np.concatenate(all_predictions, axis=0)
all_probs_pos   = np.concatenate(all_probs_pos, axis=0)

metrics_df = pd.DataFrame(all_metrics)
display(metrics_df)

print("\n===== Mean ± Std across folds =====")
for col in ["accuracy", "precision", "recall", "f1"]:
    print(f"{col:10s}: {metrics_df[col].mean():.4f} ± {metrics_df[col].std():.4f}")

metrics_df.to_csv("cv_metrics_summary.csv", index=False)
print("\nSaved to cv_metrics_summary.csv")

cm = confusion_matrix(all_true_labels, all_predictions)
acc_micro = (cm.trace()) / cm.sum()
print(f"\nMicro accuracy (all folds combined): {acc_micro:.4f}\n")
print(classification_report(all_true_labels, all_predictions, digits=4))

plt.figure(figsize=(4, 4))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    xticklabels=["Pred 0", "Pred 1"],
    yticklabels=["True 0", "True 1"],
)
plt.title("Overall Confusion Matrix (all folds)")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.tight_layout()
plt.show()

precision, recall, _ = precision_recall_curve(all_true_labels, all_probs_pos)
ap = average_precision_score(all_true_labels, all_probs_pos)

plt.figure(figsize=(5, 5))
plt.plot(recall, precision, label=f"All folds (AP = {ap:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve (5-fold combined)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

print(f"Combined Average Precision: {ap:.4f}")


# **Train Final Model**

In [ ]:
print("--- Training Final Model on All Data ---")

set_seed(42)

final_train_df = df_master
final_train_dataset = Dataset.from_pandas(final_train_df).with_transform(transform)

final_model = DeiTForImageClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

final_training_args = TrainingArguments(
    output_dir="final_deit_model_training",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    eval_strategy="no",
    save_strategy="epoch",
    num_train_epochs=15,
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_ratio=0.05,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    use_mps_device=torch.backends.mps.is_available(),
    optim="adamw_torch",
    report_to="none",
)

final_trainer = CustomTrainer(
    model=final_model,
    args=final_training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=final_train_dataset,
    tokenizer=feature_extractor,
    class_weights=class_weights_tensor,
)

final_trainer.train()

final_export_dir = "final_deit_model"
os.makedirs(final_export_dir, exist_ok=True)

final_trainer.save_model(final_export_dir)
feature_extractor.save_pretrained(final_export_dir)
torch.save({"id2label": id2label, "label2id": label2id},
           os.path.join(final_export_dir, "label_map.pt"))

print(f"--- Final Model Saved to: {final_export_dir} ---")

shutil.rmtree("final_deit_model_training")


# **Try final the model**

In [ ]:
model = DeiTForImageClassification.from_pretrained("final_deit_model")
processor = DeiTImageProcessor.from_pretrained("final_deit_model")


In [ ]:
clf = pipeline(
    "image-classification",
    model= "final_deit_model",
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
img_path = "/content/Dataset/1/DM004_M_R.png"

print(clf(img_path, top_k=2))

# **Save all content to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, shutil, pathlib

SAVE_DIR = "/content/drive/MyDrive/colab_backups_deit"
SRC_ROOT = "/content"

folder_prefixes = ("export_deit_fold_", "training_deit_fold_")
extra_folders   = {"Dataset", "sample_data", "final_deit_model"}

pathlib.Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)

for name in os.listdir(SRC_ROOT):
    src_path = os.path.join(SRC_ROOT, name)
    if os.path.isdir(src_path):
        if name.startswith(folder_prefixes) or name in extra_folders:
            dst_path = os.path.join(SAVE_DIR, name)
            print(f"Copying folder {src_path} -> {dst_path}")
            shutil.copytree(src_path, dst_path, dirs_exist_ok=True)

csv_src = os.path.join(SRC_ROOT, "cv_metrics_summary.csv")
if os.path.exists(csv_src):
    csv_dst = os.path.join(SAVE_DIR, "cv_metrics_summary.csv")
    print(f"Copying file {csv_src} -> {csv_dst}")
    shutil.copy2(csv_src, csv_dst)
else:
    print("cv_metrics_summary.csv not found in /content")

print("Done!")


# **Grad-Cam**

In [ ]:
!pip install grad-cam

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

from transformers import DeiTImageProcessor, DeiTForImageClassification
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
MODEL_FOLDER_PATH = "final_deit_model"
IMAGE_PATH_1 = "img/[1] DFU/DM115_F_R.png"  
IMAGE_PATH_2 = "img/[1] DFU/DM115_F_L.png" 
TARGET_CLASS_INDEX = 1


In [ ]:
def reshape_transform(tensor, height=14, width=14):
    result = tensor[:, 2:, :].reshape(
        tensor.size(0),
        height,
        width,
        tensor.size(2)
    )
    result = result.permute(0, 3, 1, 2)
    return result

class HuggingfaceToTensorModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super(HuggingfaceToTensorModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).logits


In [ ]:
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else ("cuda" if torch.cuda.is_available() else "cpu")
)
print("Using device:", device)

In [ ]:
processor = DeiTImageProcessor.from_pretrained(MODEL_FOLDER_PATH)
hf_model = DeiTForImageClassification.from_pretrained(MODEL_FOLDER_PATH)

model = HuggingfaceToTensorModelWrapper(hf_model).to(device).eval()

target_layers = [model.model.deit.encoder.layer[-1].layernorm_before]

cam = GradCAM(
    model=model,
    target_layers=target_layers,
    reshape_transform=reshape_transform
)

targets = [ClassifierOutputTarget(TARGET_CLASS_INDEX)]

image_paths = [IMAGE_PATH_1, IMAGE_PATH_2]
rgb_images = []          
cam_visualizations = []  

In [ ]:
for p in image_paths:
    pil_img = Image.open(p).convert("RGB")

    img_np = np.array(pil_img).astype(np.float32) / 255.0 
    rgb_images.append(img_np)

    inputs = processor(images=pil_img, return_tensors="pt")
    input_tensor = inputs["pixel_values"].to(device)  

    grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :] 

    H, W, _ = img_np.shape
    grayscale_cam_resized = cv2.resize(grayscale_cam, (W, H))

    cam_vis = show_cam_on_image(img_np, grayscale_cam_resized, use_rgb=True)
    cam_visualizations.append(cam_vis)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(7, 4))

fig.suptitle(
    f"Grad-CAM Visualization (Target Class: {TARGET_CLASS_INDEX})",
    fontsize=16,
    y=1.03
)

axes[0].imshow(rgb_images[0])
axes[0].set_title("Original Right Foot", fontsize=10, pad=4)
axes[0].axis('off')

axes[1].imshow(rgb_images[1])
axes[1].set_title("Original Left Foot", fontsize=10, pad=4)
axes[1].axis('off')

axes[2].imshow(cam_visualizations[0])
axes[2].set_title("Grad-CAM Right Foot", fontsize=10, pad=4)
axes[2].axis('off')

axes[3].imshow(cam_visualizations[1])
axes[3].set_title("Grad-CAM Left Foot", fontsize=10, pad=4)
axes[3].axis('off')

fig.subplots_adjust(
    wspace=0.05,
    left=0.03,
    right=0.97,
    top=0.85,
    bottom=0.05
)

plt.show()
